# Link the merchant+transactios (by postcode) to persona data
## Pyspark set up

In [1]:
# import libraries
from gettext import npgettext
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import numpy as np
from statistics import mean, stdev

import json

In [2]:
# setup spark
spark = (
    SparkSession.builder.appName("aggregate data for first 3 final model variables")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "15g")
    .getOrCreate()
)

22/10/06 11:41:43 WARN Utils: Your hostname, Lis-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.13.47.249 instead (on interface en0)
22/10/06 11:41:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/06 11:41:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/06 11:41:44 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/10/06 11:41:44 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/10/06 11:41:44 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/10/06 11:41:44 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
22/10/06 11:41:44 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.


## Read in data

In [3]:
postcode_df = spark.read.csv("../data/curated/Grouping.csv", header = "true")
persona_df = spark.read.csv("../data/curated/abs_cleaned.csv",  header = "true")

In [4]:
persona_df = persona_df.withColumnRenamed("_c0","postcode")

## Join persona and transaction tables together

In [5]:
cond = ["postcode"]
total_df = postcode_df.join(persona_df,cond, "left")

## Groupby and aggregation
#### For each required persona attribute, times the postcode attribute and proportion of postcode for each merchant together

In [6]:
listofmult = total_df.columns[6:]

In [7]:
total_df = total_df.withColumn("arr", F.struct(*[(F.col(x)*F.col('proportion_of_postcode')).alias(x) for x in listofmult]))\
  .select("postcode","merchant_abn","Fortnight","count_postcode_merchant","total_order_per_fornight","arr.*")

22/10/06 11:41:49 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


### For all persona attributes, sum postcode attribute proportion that belongs to each merchants

In [8]:
exprs = {x: "sum" for x in listofmult}
final_df1 = total_df.groupBy("merchant_abn","Fortnight").agg(exprs)

In [9]:
final_df1.toPandas().to_csv('../data/curated/Postcode_Persona.csv', index = False)

22/10/06 11:41:50 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , mean_Age, min_Age, q1_Age, median_Age, q3_Age, max_Age, Millen_Z%, Oceania and Antarctica, North-West Europe, Southern and Eastern Europe, North Africa and the Middle East, South-East Asia, North-East Asia, Southern and Central Asia, Americas, Sub-Saharan Africa, mean_Salary, median_Salary, min_Salary, q1_Salary, q3_Salary, max_Salary, Managers_%, Professionals_%, Technicians and Trades Workers_%, Community and Personal Service Workers_%, Clerical and Administrative Workers_%, Sales Workers_%, Machinery Operators and Drivers_%, Labourers_%, Total
 Schema: _c0, mean_Age, min_Age, q1_Age, median_Age, q3_Age, max_Age, Millen_Z%, Oceania and Antarctica, North-West Europe, Southern and Eastern Europe, North Africa and the Middle East, South-East Asia, North-East Asia, Southern and Central Asia, Americas, Sub-Saharan Africa, mean_Salary, median_Salary, min_Salary, q1_Salary, q3_Salary, max_Salary, 